In [24]:
from google.cloud import language_v1
import os
import re

In [25]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './cueback-app-e91f3f8c0155.json'

In [26]:
client = language_v1.LanguageServiceClient()

In [27]:
def extract_entity(text):
    global client
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    return client.analyze_entities(request={'document': document})

def extract_sentiment(text):
    global client
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    return client.analyze_sentiment(request={'document': document})

In [64]:
def entity_sentiment(text):
    r1 = extract_entity(text)
    entities_sentiment={}
    for e in r1.entities:
        if str(e.type_) in ['Type.PERSON', 'Type.ORGANIZATION', 'Type.LOCATION', 'Type.DATE']:
            sentiments = []
            for mention in e.mentions:
                try:
                    sentences = re.findall(r"([^.]*?"+mention.text.content+"[^.]*\.)",text)  
                    for s in sentences:
                        sentiment = extract_sentiment(s)
                        sentiments.append(sentiment.document_sentiment.score)
                except:
                    pass
            if len(sentiments) != 0:
                entities_sentiment[e.name] = sum(sentiments) / len(sentiments)
    entities_sentiment = dict(sorted(entities_sentiment.items(), key=lambda item: item[1], reverse=True))
    print(entities_sentiment)

# Tests

In [57]:
test = """In the Fall of 1971, our new DJ Trainees were very interested to know just what music they could play on KCPR. Steve Stagnero, our Program Director, visited the new DJ Training class and he was asked for his recommendations. He simply answered, "Beatles, Beach Boys and Blood, Sweat & Tears." So now we knew.

The next thing DJ trainees had to learn was the KCPR music format. It was a fairly complicated policy that divided the broadcast day into six parts and established a different balance of musical content for each part. Even today, KCPR tries to have a consistent sound from one day to the next with regular format programs entitled The Breakfast Club, Afternoon Delight, The Comedown and The Lounge. I don't know how KCPR manages their musical format today but, back in 1971, we had Format Sheets.

From hour to hour, the Format Sheets told us how many "Popular", "Rock" (not the same thing in a world with both Andy Williams and The Who), "Oldies" and "Heavy" songs to play. We drew Format Sheets for our shift and filled them out as we played the required combinations of mandatory and optional songs every hour. I do not know if anyone ever looked at them but I don't recall them sticking around very long.

I only had a few air shifts after completing DJ Training in the Fall of 1971 so I only filled out a few of the Format Sheets and don't recall seeing them when we returned for the Winter Quarter in 1972. Fortunately, a few of us that were doing air shifts in the Fall of 1971 are here and hopefully they will tell us more about how KCPR managed music on the air."""

entity_sentiment(test)

{'DJ Trainees': 0.800000011920929, 'Steve Stagnero': 0.6000000238418579, '1971': 0.2500000037252903, 'KCPR': 0.22000000327825547, 'parts': 0.10000000149011612, 'Format Sheets': 0.02500000037252903, 'few': 0.0, 'The Lounge': 0.0, 'DJ trainees': -0.10000000149011612, 'world': -0.10000000149011612, 'Blood, Sweat & Tears': -0.10000000149011612, 'Beach Boys': -0.10000000149011612, 'The Who': -0.10000000149011612, 'Andy Williams': -0.10000000149011612, 'Winter Quarter': -0.20000000298023224, '1972': -0.20000000298023224, 'anyone': -0.4000000059604645}


In [58]:
test = """After reading Jim Stueck's senior project, I got the impression that these General Polices were rooted in what Alan Holmes and Jim Stueck had started in 1968-69 but the policies had to evolve to address the growing size of KCPR's staff, its longer operating hours and its expanding studio and transmitter facilities. Writing these policies down and teaching them to the DJ trainees would help everyone work together and keep the student run radio station out of trouble. However, the efficient use of ditto masters in the preparation of this document definitely reminded me of Woody Goulart."""

entity_sentiment(test)

{'radio station': 0.10000000149011612, 'student': 0.10000000149011612, 'everyone': 0.10000000149011612, 'DJ trainees': 0.10000000149011612, 'Jim Stueck': 0.0, 'Alan Holmes': 0.0, 'studio': 0.0, 'staff': 0.0, 'facilities': 0.0, 'KCPR': 0.0, 'Woody Goulart': 0.0}


In [59]:
test ="""This is a test of the Cueback Collaboration System. Anyone listed as a collaborator to this story can add their memories directly to it rather than just make comments. And that is going to be necessary because, as big an event as this was, I don't remember much about it other than who else was there and what was reported in the Mustang Daily.

It was January 18, 1973 and the rain has been falling heavily over San Luis Obispo. San Luis Creek had backed up which resulted in significant flooding downtown, including at the corner of Osos and Marsh Streets where KSLY had their studios below grade, in the basement of the San Luis Savings and Loan. The flood was a disaster for them. For the second time since 1969, KSLY had to evacuate its studios and take all of the studio equipment they could salvage to the safety of their transmitter site.

Another transmitter site was about to play a role in the saga of the flood of 1973. KCPR maintained its transmitter facilities in KVEC's transmitter site on Radio Hill, which is on the Cal Poly campus. As the flooding got worse on campus, power going to affected buildings, including the Graphic Arts Building, was being turned off. On the morning of Jan 18th, KCPR's studios became blacked out. But not KVEC. They were the primary Emergency Broadcast System station for San Luis Obispo and the federal government had provided funds for them to install emergency power generators at their studios and at their transmitter site. So whether the power on the Cal Poly campus was shut off or not, KVEC was still able to generate their own power and stay on the air.

KCPR took advantage of KVEC's situation to take our portable audio console up to the transmitter site we shared with them and resumed live programming from there. Steve Ruegnitz and Eric Dausman loaded the portable audio console in Eric's '57 Chevy wagon and hauled it up to Radio Hill where they were joined by Dean Opperman, Al Lindley, Steve Witten and Ron Margaretten. I recall dropping by in my '65 VW but didn't stay long so I hope my friends can collaborate on happened up there.

According to the story in the Mustang Daily, KCPR conducted broadcasting on Radio Hill from 10:15AM to 5PM on Jan 18th. Check out the pictures of the flooding around town starting on page 3: http://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?article=2942&context=studentnewspaper"""
entity_sentiment(test)

{'collaborator': 0.4000000059604645, 'Anyone': 0.4000000059604645, 'Radio Hill': 0.0, 'government': 0.0, 'station': 0.0, 'transmitter facilities': 0.0, 'Emergency Broadcast System': 0.0, 'friends': 0.0, 'Eric Dausman': 0.0, 'Steve Ruegnitz': 0.0, 'Chevy': 0.0, 'Dean Opperman': 0.0, 'Al Lindley': 0.0, 'Steve Witten': 0.0, 'Ron Margaretten': 0.0, 'transmitter site': -0.04000000059604645, 'Mustang Daily': -0.10000000149011612, 'KCPR': -0.17499999701976776, 'KVEC': -0.17500000074505806, 'San Luis Obispo': -0.20000000298023224, 'town': -0.20000000298023224, '1969': -0.20000000298023224, '1973': -0.20000000298023224, 'campus': -0.2666666756073634, 'KSLY': -0.30000000447034836, 'studios': -0.32499999925494194, 'Jan 18th': -0.3499999940395355, 'Jan': -0.36666666467984516, 'San Luis Creek': -0.4000000059604645, 'Marsh Streets': -0.4000000059604645, 'corner': -0.4000000059604645, 'flooding downtown': -0.4000000059604645, 'basement': -0.4000000059604645, 'January 18, 1973': -0.4000000059604645, '

In [60]:
test = """In 2008, Scott Larsen wrote in the kcpr-grads Yahoo Group: "I made that sign out of masonite, painted it black and prayed it would survive the weekend... funny that it's still there. After the first Poly Royal I remember taking it down (just laying it on the roof), then it went back up for graduation and stayed a while, then I think for the next poly royal some campus official finally noticed it and I promised it was just for Poly Royal and we'd take it down after the weekend... up down up down a few times then it just stuck I guess.

Jon Jaeger originally posted this photo on the KCPR Alumni Group back in 2014. Frank Warren remembers the sign went up for Poly Royal in 1987. That's station manger Scott Larsen with his arms folded. Jon is sitting on the roof."""

entity_sentiment(test)

{'Yahoo Group': 0.0, 'KCPR Alumni Group': 0.0, 'Frank Warren': 0.0, '2008': 0.0, '2014': 0.0, '1987': 0.0, 'Royal': -0.10000000149011612, 'Poly Royal': -0.10000000149011612, 'Jon Jaeger': -0.10000000397364299, 'campus official': -0.20000000298023224, 'Scott Larsen': -0.24000000357627868}


# Tests II

In [61]:
import json

In [62]:
f = open('./data/notesdump.json')
data = json.load(f)
tests = []
for test in data['value']:
    try:
        tests.append(test['text'])
    except:
        pass
sample = tests[:10]

In [63]:
for test in sample:
    print(test)
    entity_sentiment(test)
    print('\n')

Mr. Diresta's name was mentioned in the Birmingham Times in an article about The Birmingham Medical Center.  He apparently donated an undisclosed amount of money to assist with the new surgical wing.  He prefers to give to healthcare related organizations.
{'Birmingham Times': 0.0, 'The Birmingham Medical Center': 0.0, 'organizations': 0.0, 'Diresta': -0.05000000074505806}


From Atlanta to Birmingham for school.  His girlfriend (now wife) was already living in Birmingham at the time.
{'Atlanta': 0.20000000298023224, 'school': 0.20000000298023224, 'Birmingham': 0.05000000074505806, 'girlfriend': -0.10000000149011612}


The dinner that was held for the Medical School was partially sponsored by Mr. Regis Young, Christopher's father.  Both expressed interest in attending all alumni functions.  Send all correspondence to home address.
{'Regis Young': 0.0, 'Christopher': 0.0, 'Medical School': -0.20000000298023224}


I went to Culver Memorial to visit Gretchen Smith's neice and saw Dr. Youn

# Test III

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('./data/archive/data_science_for_fundraising_contact_reports.csv')

In [15]:
df_filtered = df[~(df['Summary']=='not interested in the University')]
sample = df_filtered.head(50)

In [16]:
for _, row in sample.iterrows():
    test = row['Summary']
    print(test)
    entity_sentiment(test)
    print('\n')

Sent Dominic an email asking for an update if he would like to move ahead with a gift to establish the full Graduate Student Honors Program based on how pleased he was with the intern program
{}


Emailed Dr. Jonathan Winters, who have ties to Dr. Bendrich, regarding a gift to fund a Student Honors Program based on his verbal interest
{'Jonathan Winters': 0.0, 'Bendrich': 0.0, 'ties': 0.0}


Sent Belinda an upgrade brochure, appeal letter and envelope, as well as a handwritten note asking for her to become a leadership annual fund donor with a gift of $1,000
{}


$50k to extend Kendrick Memorial Undergraduate Scholarship and create Guiles Memorial Student Scholarship based on board meeting and verbal commtiment
{}


Sent upgrade brochure asking Jeffrey and Julie to become a leadership annual fund donor by making a gift of $1,000 or more, and said I'd be happy to send information about our Leadership Society
{}


Sent Marsha an leadership annual fund upgrade brochure with a handwritten 

ResourceExhausted: 429 Quota exceeded for quota metric 'Requests' and limit 'Requests per minute' of service 'language.googleapis.com' for consumer 'project_number:298180869348'.